In [25]:
from pip._internal import main as pip

try:
    import requests
except ImportError:
    pip(['install', 'request'])
    import requests
    
try:
    import urllib
except ImportError:
    pip(['install', 'urllib'])
    import urllib

try:
    import pandas
except ImportError:
    pip(['install', 'pandas'])
    import pandas

try:
    from IPython.display import display, SVG, HTML, Image
except ImportError:
    pip(['install', 'IPython'])
    from IPython.display import display, SVG, HTML
    



In [31]:
url = 'https://lazar.prod.openrisknet.org/endpoint'
headers = {'accept': 'application/json',
           'Content-Type': 'application/x-www-form-urlencoded'}

r1 = requests.get(url, headers=headers)

print("LAZAR Status code GET endpoints: {0}".format(r1.status_code))
if r1.status_code == 200:
    endpoints = r1.json()

LAZAR Status code GET endpoints: 200


In [47]:
data = []

def extractSpecies(speciesList):
    for item in speciesList:
        for species, url in item.items():
            yield species, url            
            
for endpoint in endpoints:
    url = 'https://lazar.prod.openrisknet.org/endpoint/' + endpoint
    r2 = requests.get(url, headers=headers);

    if r2.status_code == 200:
        for species, url in extractSpecies(r2.json()):
            data.append([endpoint, species, url])

df = pandas.DataFrame(data, columns=["Endpoint", "Species", "URL"])

df


,Endpoint,Species,URL
0,Blood Brain Barrier Penetration,Human,https://lazar.prod.openrisknet.org/model/5ae2dd885f1c2d01323270ee
1,Carcinogenicity,Rat,https://lazar.prod.openrisknet.org/model/5ae2de075f1c2d0132327658
2,Carcinogenicity,Rodents (multiple species/sites),https://lazar.prod.openrisknet.org/model/5ae2e2425f1c2d0132328730
3,Carcinogenicity,Mouse,https://lazar.prod.openrisknet.org/model/5ae2e2a55f1c2d0132328867
4,Carcinogenicity,Mouse (TD50),https://lazar.prod.openrisknet.org/model/5ae2ff635f1c2d0132329a54
5,Carcinogenicity,Rat (TD50),https://lazar.prod.openrisknet.org/model/5ae3034f5f1c2d0132329c82
6,Mutagenicity,Salmonella typhimurium,https://lazar.prod.openrisknet.org/model/5ae2e1cd5f1c2d0132328594
7,Lowest observed adverse effect level (LOAEL),Rat,https://lazar.prod.openrisknet.org/model/5ae2e9225f1c2d0132328d67
8,Acute toxicity,Daphnia magna,https://lazar.prod.openrisknet.org/model/5ae2eda05f1c2d0132328fde
9,Acute toxicity,Fathead minnow,https://lazar.prod.openrisknet.org/model/5ae3097a5f1c2d0132329f40


In [48]:
r3 = requests.post(df['URL'][1], 
                   data={'identifier': "CCCCCCCC(=O)OC(COC(=O)CCCCCCC)COC(=O)CCCCCCC"},
                   headers=headers);
print("LAZAR Status code POST predict SMILES: {0}".format(r3.status_code))
if r3.status_code == 200:
    prediction = r3.json()

LAZAR Status code POST predict SMILES: 200


In [49]:
df_probabilities = pandas.DataFrame(prediction['prediction']['probabilities'], index=[prediction['compound']['smiles']])

print("LAZAR prediction result:")

df_probabilities

LAZAR prediction result:


,carcinogenic,non-carcinogenic
CCCCCCCC(=O)OC(COC(=O)CCCCCCC)COC(=O)CCCCCCC,0.170328,0.401101


In [50]:
pandas.set_option('display.max_colwidth', -1)

prediction_neighbors = pandas.DataFrame(prediction['prediction']['neighbors']).sort_values(['similarity'], ascending=False)
#print(prediction_neighbors)
prediction_neighbors.index = range(0,len(prediction_neighbors))
#print(prediction_neighbors)

df_neighbors = pandas.DataFrame(columns=['Image', 'SMILES', 'Measurement', 'Similarity'])

idx = 0
for id in prediction_neighbors['id']:
    compoundreq = requests.get('https://lazar.prod.openrisknet.org/substance/'+id, headers={'accept':'application/json'})
    compound = compoundreq.json()
    c = compound['compound']
    smiles = c['smiles']
    inchi = c['inchi']
    esc_inchi = urllib.parse.quote(inchi)
    req=requests.get('https://lazar.prod.openrisknet.org/compound/'+esc_inchi, headers={'accept':'image/svg+xml'})
    svg=req.content
    df_neighbors = df_neighbors.append({'Image': svg, 'SMILES': smiles, 'Measurement': prediction_neighbors.at[idx, 'measurement'], 'Similarity': prediction_neighbors.at[idx, 'similarity']}, ignore_index=True)
    idx = idx+1


HTML(df_neighbors.to_html(escape=False, formatters=dict(status=svg)))
